In [13]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Polygon
from IPython.display import HTML

# ==========================================
# 1. MATH & PHYSICS HELPERS
# ==========================================

def get_rotation_matrix(theta):
    """Returns 3x3 rotation matrix for 2D plane (Z-axis rotation)."""
    c, s = np.cos(theta), np.sin(theta)
    return np.array([
        [c, -s, 0],
        [s,  c, 0],
        [0,  0, 1]
    ])

def get_transform(x, y, theta):
    """Returns 3x3 homogeneous transformation matrix (Body -> Inertial)."""
    c, s = np.cos(theta), np.sin(theta)
    return np.array([
        [c, -s, x],
        [s,  c, y],
        [0,  0, 1]
    ])

# ==========================================
# 2. DYNAMIC ACKERMANN MODEL
# ==========================================

class DynamicAckermann:
    def __init__(self, mass=1500, Iz=2500, L=2.5, W=1.6):
        # Parameters
        self.m = mass          # kg
        self.Iz = Iz           # kg*m^2
        self.L = L             # Wheelbase (m)
        self.W = W             # Track width (m)
        self.r_wheel = 0.3     # Wheel radius (m)
        self.Ca = 15000        # Cornering Stiffness (N/rad)
        
        # State: [x, y, theta, vx, vy, omega]
        self.state = np.zeros(6) 
        
        # Wheel positions relative to Center of Gravity (CoG)
        self.wheel_offsets = {
            'FL': np.array([L/2, W/2]),
            'FR': np.array([L/2, -W/2]),
            'RL': np.array([-L/2, W/2]),
            'RR': np.array([-L/2, -W/2])
        }

    def get_matrices(self, q):
        """Returns M, C, G, B matrices for the equation: M*q_dd + C*q_d + G = B*u"""
        theta = q[2]
        
        # M(q): Mass Matrix
        M = np.diag([self.m, self.m, self.Iz])
        
        # C(q, q_dot): Coriolis (Zero for Cartesian Newton-Euler on flat plane)
        C = np.zeros((3, 3))
        
        # G(q): Gravity (Zero on flat plane)
        G = np.zeros(3)
        
        # B(q): Input Map (Body Frame Forces -> World Frame)
        B = get_rotation_matrix(theta)
        
        return M, C, G, B

    def compute_forces(self, state, torques, steering_angles):
        """Calculates tire forces based on slip angles and input torques."""
        x, y, theta, vx, vy, omega = state
        
        # Transform World Velocity to Body Frame
        R_T = get_rotation_matrix(theta).T
        v_world = np.array([vx, vy, 0])
        v_body = R_T @ v_world
        vx_b, vy_b = v_body[0], v_body[1]
        
        total_Fx, total_Fy, total_Mz = 0, 0, 0
        
        for w_name, offset in self.wheel_offsets.items():
            # 1. Wheel Velocity (Rigid Body Kinematics)
            wx_vel = vx_b - omega * offset[1]
            wy_vel = vy_b + omega * offset[0]
            
            # 2. Slip Angle (alpha)
            # Avoid division by zero at low speeds
            if abs(wx_vel) < 0.1:
                alpha = 0
            else:
                alpha = steering_angles[w_name] - np.arctan2(wy_vel, wx_vel)
                
            # 3. Tire Forces
            # Longitudinal Force (from Torque)
            F_long = torques[w_name] / self.r_wheel
            # Lateral Force (Linear Tire Model)
            F_lat = self.Ca * alpha
            
            # 4. Rotate Forces to Chassis Body Frame
            delta = steering_angles[w_name]
            c_d, s_d = np.cos(delta), np.sin(delta)
            
            Fx_body = F_long * c_d - F_lat * s_d
            Fy_body = F_long * s_d + F_lat * c_d
            
            # 5. Accumulate Total Wrench
            total_Fx += Fx_body
            total_Fy += Fy_body
            total_Mz += offset[0] * Fy_body - offset[1] * Fx_body

        return np.array([total_Fx, total_Fy, total_Mz])

    def step(self, dt, torques, steering_angle):
        # 1. Unpack State
        q = self.state[0:3]
        q_dot = self.state[3:6]
        
        # 2. Get Matrices
        M, C, G, B = self.get_matrices(q)
        
        # 3. Compute Input Forces (RHS)
        steer_angles = {'FL': steering_angle, 'FR': steering_angle, 'RL': 0, 'RR': 0}
        F_body_total = self.compute_forces(self.state, torques, steer_angles)
        
        # 4. Solve Dynamics: q_ddot = inv(M) * (B*F_body - C*q_dot - G)
        RHS = B @ F_body_total - C @ q_dot - G
        q_ddot = np.linalg.inv(M) @ RHS
        
        # 5. Integrate (Euler)
        self.state[3:6] += q_ddot * dt
        self.state[0:3] += self.state[3:6] * dt
        
        return self.state

# ==========================================
# 3. SIMULATION LOOP (TEST CASES)
# ==========================================

# Setup
car = DynamicAckermann(mass=1500, Iz=2500)
dt = 0.02

# CHANGED: Increased from 12.0 to 20.0 seconds
duration = 20.0  
steps = int(duration / dt)
history = {'x': [], 'y': [], 'theta': []}

# Global Inputs
steering_input = np.radians(40) # Constant 15 deg turn

for i in range(steps):
    t = i * dt
    
    # --- TEST CASE: TORQUE SETS ---
    # Phase 1: Acceleration (0 to 4 seconds)
    if t < 2.0:
        torque_val = 250.0  # Nm (Gas)
    # Phase 2: Coasting (4s to end)
    else:
        torque_val = 0.0    # Nm (Coast/Friction)
        
    # Apply to Rear Wheels only
    torques = {'FL': 0, 'FR': 0, 'RL': torque_val, 'RR': torque_val}
    
    # Step Physics
    state = car.step(dt, torques, steering_input)
    
    # Record
    history['x'].append(state[0])
    history['y'].append(state[1])
    history['theta'].append(state[2])

# ==========================================
# 4. ANIMATION (RESTORED STYLE)
# ==========================================

fig, ax = plt.subplots(figsize=(10, 10))
ax.set_xlim(-10, 60)
ax.set_ylim(-20, 50)
ax.set_aspect('equal')
ax.grid(True)
ax.set_title("Dynamic Test: Accel (0-4s) -> Coast")

# Visual Elements
trail, = ax.plot([], [], 'b-', lw=1.5, alpha=0.6, label='Trail')
car_body = Polygon(np.zeros((4, 2)), fc='lightblue', ec='black', alpha=0.9, label='Chassis')
wheels = {
    'FL': Polygon(np.zeros((4, 2)), fc='red'), 'FR': Polygon(np.zeros((4, 2)), fc='red'),
    'RL': Polygon(np.zeros((4, 2)), fc='black'), 'RR': Polygon(np.zeros((4, 2)), fc='black')
}

# Add Patches
ax.add_patch(car_body)
for w in wheels.values(): ax.add_patch(w)
ax.legend()

# Drawing Constants
L, W = car.L, car.W
w_len, w_wid = 0.6, 0.25
TRAIL_LEN = 150

def update(i):
    # State
    x, y, theta = history['x'][i], history['y'][i], history['theta'][i]
    
    # 1. Trail
    start = max(0, i - TRAIL_LEN)
    trail.set_data(history['x'][start:i+1], history['y'][start:i+1])
    
    # 2. Robot Transform
    T = get_transform(x, y, theta)
    
    # 3. Body (Centered at CoG)
    body_pts = np.array([
        [-L/2-0.2, W/2+0.1], [-L/2-0.2, -W/2-0.1],
        [L/2+0.2, -W/2-0.1], [L/2+0.2, W/2+0.1]
    ])
    body_world = (T @ np.hstack([body_pts, np.ones((4,1))]).T).T
    car_body.set_xy(body_world[:, :2])
    
    # 4. Wheels
    for w_name, w_poly in wheels.items():
        pos_r = car.wheel_offsets[w_name]
        angle = steering_input if 'F' in w_name else 0.0
        
        # Wheel shape at origin
        w_pts = np.array([
            [-w_len/2, w_wid/2], [-w_len/2, -w_wid/2],
            [w_len/2, -w_wid/2], [w_len/2, w_wid/2]
        ])
        
        # Rotate(Steer) -> Translate(Mount) -> Transform(World)
        c, s = np.cos(angle), np.sin(angle)
        R_s = np.array([[c, -s], [s, c]])
        w_local = w_pts @ R_s.T + pos_r
        w_world = (T @ np.hstack([w_local, np.ones((4,1))]).T).T
        
        w_poly.set_xy(w_world[:, :2])
        
    return [car_body, trail] + list(wheels.values())

anim = animation.FuncAnimation(fig, update, frames=np.arange(0, len(history['x']), 2), interval=20, blit=True)
plt.close()
HTML(anim.to_html5_video())